In [ ]:
import pandas as pd
import numpy as np

filepath = "C:/Users/14344/Downloads/Fabricated_Health_Data.csv"
data = pd.read_csv(filepath)

print(data.head(3))


# Project Queries:
# is there a link to chronic disease vs smoking, chronic disease vs exercise, chronic disease vs region


In [ ]:
# Add a new row
print("length of data: ", len(data)) # original is 100, I'll add a dupe row and then drop dupe row further into project
print("The Last row was: ", data.loc[99]) # returns the last row
data.loc[len(data)] = ['P001', 58, 'Female', np.NaN, 'Non-Smoker', 'Occasional', 'Yes', 'South' ]
print("Now the last row is: ", data.loc[100])

In [ ]:
# Check for missing values
print("Missing values: \n", data.isnull().sum())

# Basic statistics
print("\n",data.describe())
print("\n",data.info())

# Distribution of categorical variables
categorical_variables = data.select_dtypes(["object", "category"])
print( 'Categorical columns are: \n', categorical_variables.columns)


age_bins = pd.cut(data['Age'],bins=range(0,100,10)) # create age groups for analysis #Use cut to segment and sort data values into bins. 

print("\n The Age of the patients are \n",age_bins.value_counts().sort_index())
print("\n The Regions reported are \n",data['Region'].value_counts())
print("\n The Gender distribution of patients is \n",data['Gender'].value_counts())


In [ ]:
# Check for duplicates
print("Duplicate rows: ", data.duplicated().sum()) # data[data.duplicated()] will show the rows themselves

# Remove duplicates if any
data = data.drop_duplicates()

# Example: Replace missing BMI values with the mean (if missing values existed)
print("Total null cells in the BMI column is: ", data['BMI'].isnull().sum()) #checks if there are null values in BMI column

if data["BMI"].isnull().sum()>0:
    data["BMI"].fillna(data["BMI"].mean(), inplace=True) # if you do this without inplace, no effect on the table 

# Check the last row, has it's null replaced with the average of BMIs  
print(data.loc[100])



In [ ]:
import sqlite3
import pandas as pd

# Create a SQLite database in memory and load the data
#initiate a sql connection strapped to memory ( temp)
conn = sqlite3.connect(":memory:") 

# change the dataframe into a sql table and name it health data, use the connection created above
data.to_sql( name= 'health_data', con = conn, if_exists='replace', index = False) 


# Example Query: Average BMI by Region
# write the query
query_avg_bmi_region = """
SELECT Region, ROUND(AVG(BMI),2) as Average_BMI
FROM health_data
GROUP BY Region
ORDER BY Average_BMI DESC
"""

# read the query
avg_bmi = pd.read_sql_query(sql = query_avg_bmi_region, con = conn)
print(avg_bmi)



# Example Query: Percentage of Smokers by Region
query_smokers_region = """

SELECT Region, 
    ROUND(SUM(IIF(Smoking_status = 'Smoker', 1, 0)) * 100.0 / COUNT(*), 2) AS Smoking_Percentage 
FROM health_data 
GROUP BY Region 
ORDER BY Smoking_Percentage DESC;
"""

smokers_percentage= pd.read_sql_query(query_smokers_region, con=conn)
print(smokers_percentage)


In [ ]:
import matplotlib.pyplot as plt

# Bar chart: Average BMI by Region
print(avg_bmi)
plt.bar(avg_bmi['Region'], avg_bmi['Average_BMI'], color = 'skyblue')
plt.title('Average BMI by Region')
plt.xlabel('Region')
plt.ylabel('Average BMI')
plt.show() 


# Bar chart: Smoking Percentage by Region
print(smokers_percentage)

plt.bar(smokers_percentage['Region'], smokers_percentage['Smoking_Percentage'], color= 'salmon')
plt.title("Smoking Percentage by Region")
plt.xlabel("Region")
plt.ylabel("Percentage of Smokers")
plt.show()
